# Proceo de Limpieza

In [10]:
# imports
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
data_test = pd.read_csv('../data/test_format2.csv')
data_train = pd.read_csv('../data/train_format2.csv')

# La limpieza de datos se realiza en ambos datasets

# Validaciones rapidas para entender como estan los datos

## Visión rápida: tamaños, tipos y nulos

In [4]:
# Tamaños y tipos
for name, df in {"train": data_train, "test": data_test}.items():
    print(f"=== {name.upper()} ===")
    print("shape:", df.shape)
    print(df.dtypes)
    print()

=== TRAIN ===
shape: (7030723, 6)
user_id           int64
age_range       float64
gender          float64
merchant_id       int64
label             int64
activity_log     object
dtype: object

=== TEST ===
shape: (7027943, 6)
user_id           int64
age_range       float64
gender          float64
merchant_id       int64
label           float64
activity_log     object
dtype: object



In [5]:
# Conteo de nulos por columna
print("=== NULLS TRAIN ===")
print(data_train.isna().sum().sort_values(ascending=False))
print("\n=== NULLS TEST ===")
print(data_test.isna().sum().sort_values(ascending=False))

=== NULLS TRAIN ===
gender          61712
age_range       19380
activity_log     2975
user_id             0
merchant_id         0
label               0
dtype: int64

=== NULLS TEST ===
label           261477
gender           63250
age_range        19420
activity_log      3006
user_id              0
merchant_id          0
dtype: int64


## Variables clave: distribución de label (train), age_range, gender

In [6]:
# Distribución de label en TRAIN (incluye -1 si hay)
print("=== LABEL value_counts (TRAIN) ===")
print(data_train['label'].value_counts(dropna=False))

# Edad y género (TRAIN y TEST)
def vc_with_na(s):
    return s.value_counts(dropna=False).sort_index()

print("\n=== age_range (TRAIN) ===")
print(vc_with_na(data_train['age_range']))

print("\n=== age_range (TEST) ===")
print(vc_with_na(data_test['age_range']))

print("\n=== gender (TRAIN) ===")
print(vc_with_na(data_train['gender']))

print("\n=== gender (TEST) ===")
print(vc_with_na(data_test['gender']))


=== LABEL value_counts (TRAIN) ===
label
-1    6769859
 0     244912
 1      15952
Name: count, dtype: int64

=== age_range (TRAIN) ===
age_range
0.0    1351842
1.0        286
2.0     731938
3.0    1913722
4.0    1459923
5.0     752927
6.0     655922
7.0     124493
8.0      20290
NaN      19380
Name: count, dtype: int64

=== age_range (TEST) ===
age_range
0.0    1345565
1.0        260
2.0     733323
3.0    1916611
4.0    1460542
5.0     752608
6.0     650358
7.0     128644
8.0      20612
NaN      19420
Name: count, dtype: int64

=== gender (TRAIN) ===
gender
0.0    5101730
1.0    1618110
2.0     249171
NaN      61712
Name: count, dtype: int64

=== gender (TEST) ===
gender
0.0    5062667
1.0    1643382
2.0     258644
NaN      63250
Name: count, dtype: int64


## activity_log: calidad básica (vacíos, longitud, ejemplos)

In [7]:
# Longitud de log por registro (nº de interacciones por fila)
def count_interactions(x):
    if pd.isna(x) or x == '':
        return 0
    return len(str(x).split('#'))

for name, df in {"train": data_train, "test": data_test}.items():
    df['activity_len'] = df['activity_log'].apply(count_interactions)
    print(f"=== {name.upper()} activity_len ===")
    print(df['activity_len'].describe())
    print("Ceros (sin interacciones):", (df['activity_len'] == 0).sum())
    print()

# Muestra de 3 filas con logs no vacíos
print("=== EJEMPLOS activity_log (TRAIN) ===")
print(data_train.loc[data_train['activity_len']>0, 'activity_log'].head(3).to_list())

=== TRAIN activity_len ===
count    7.030723e+06
mean     3.894925e+00
std      1.214883e+01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      6.963000e+03
Name: activity_len, dtype: float64
Ceros (sin interacciones): 2975

=== TEST activity_len ===
count    7.027943e+06
mean     3.905862e+00
std      1.275945e+01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      7.688000e+03
Name: activity_len, dtype: float64
Ceros (sin interacciones): 3006

=== EJEMPLOS activity_log (TRAIN) ===
['408895:1505:7370:1107:0', '17235:1604:4396:0818:0#954723:1604:4396:0818:0#275437:1604:4396:0818:0#548906:1577:4396:1031:0#368206:662:4396:0818:0#480007:1604:4396:0818:0#954723:1604:4396:0818:0#236488:1505:4396:1024:0', '231901:662:2758:0818:0#231901:662:2758:0818:0#108465:662:2758:0820:0#231901:662:2758:0820:0#231901:662:2758:0820:0#840446:1142:2758:0820:0#231901:662:2758:0819:0']


## Parseo mínimo del activity_log: validar formato y action_type

In [9]:
def parse_records(log):
    """
    Devuelve lista de tuplas (item_id, category_id, brand_id, time_stamp, action_type)
    o [] si vacío/malformado.
    """
    if pd.isna(log) or log == '':
        return []
    recs = []
    for rec in str(log).split('#'):
        parts = rec.split(':')
        if len(parts) == 5:
            recs.append(tuple(parts))
        else:
            recs.append(None)
    return recs

def sample_action_types(df, n_rows=2000):
    atypes = Counter()
    malformed = 0
    for log in df['activity_log'].head(n_rows):
        recs = parse_records(log)
        for r in recs:
            if r is None:
                malformed += 1
            else:
                atypes.update([r[4]])
    return atypes, malformed

print("=== ACTION TYPES (TRAIN, muestra 2000 filas) ===")
atypes_train, malformed_train = sample_action_types(data_train, 2000)
print("malformados (TRAIN muestra):", malformed_train)
print("action_types (TRAIN muestra):", atypes_train.most_common(20))

print("\n=== ACTION TYPES (TEST, muestra 2000 filas) ===")
atypes_test, malformed_test = sample_action_types(data_test, 2000)
print("malformados (TEST muestra):", malformed_test)
print("action_types (TEST muestra):", atypes_test.most_common(20))

=== ACTION TYPES (TRAIN, muestra 2000 filas) ===
malformados (TRAIN muestra): 0
action_types (TRAIN muestra): [('0', 6551), ('3', 503), ('2', 463)]

=== ACTION TYPES (TEST, muestra 2000 filas) ===
malformados (TEST muestra): 0
action_types (TEST muestra): [('0', 6860), ('3', 601), ('2', 417)]


## Validación de time_stamp: rango temporal, parseo y errores

In [11]:
def extract_timestamps(df, n_rows=2000):
    tvals = []
    bad = 0
    for log in df['activity_log'].head(n_rows):
        recs = parse_records(log)
        for r in recs:
            if r is None:
                continue
            tvals.append(r[3])
    ts = pd.to_datetime(pd.Series(tvals), errors='coerce', infer_datetime_format=True)
    bad = ts.isna().sum()
    return ts, bad

print("=== TIMESTAMPS (TRAIN muestra 2000 filas) ===")
ts_train, bad_train = extract_timestamps(data_train, 2000)
print("parseables:", ts_train.notna().sum(), "no parseables:", bad_train)
if ts_train.notna().any():
    print("min:", ts_train.min(), "max:", ts_train.max())

print("\n=== TIMESTAMPS (TEST muestra 2000 filas) ===")
ts_test, bad_test = extract_timestamps(data_test, 2000)
print("parseables:", ts_test.notna().sum(), "no parseables:", bad_test)
if ts_test.notna().any():
    print("min:", ts_test.min(), "max:", ts_test.max())

=== TIMESTAMPS (TRAIN muestra 2000 filas) ===
parseables: 0 no parseables: 7517

=== TIMESTAMPS (TEST muestra 2000 filas) ===
parseables: 0 no parseables: 7878


C:\Users\thiag\AppData\Local\Temp\ipykernel_30120\2817643149.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  ts = pd.to_datetime(pd.Series(tvals), errors='coerce', infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_30120\2817643149.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(pd.Series(tvals), errors='coerce', infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_30120\2817643149.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/p

## Duplicados potenciales (por par usuario–comerciante)

In [12]:
print("=== Duplicados exactos ===")
print("TRAIN:", data_train.duplicated().sum())
print("TEST:", data_test.duplicated().sum())

def dup_pairs(df):
    if all(c in df.columns for c in ['user_id', 'merchant_id']):
        return df.duplicated(subset=['user_id','merchant_id']).sum()
    return None

print("\n=== Duplicados por (user_id, merchant_id) ===")
print("TRAIN:", dup_pairs(data_train))
print("TEST:", dup_pairs(data_test))

=== Duplicados exactos ===
TRAIN: 0
TEST: 0

=== Duplicados por (user_id, merchant_id) ===
TRAIN: 0
TEST: 0
